# Aufgabe 2: Generative Adversarial Networks

In dieser Aufgabe wollen wir ein *DCGAN* auf Basis des CIFAR10 Datensatzes trainieren. 

In [ ]:
import os
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.utils as vutils
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from tqdm.notebook import tqdm

## Laden der Bilddaten


In [ ]:
img_size = 64
#batch_size=64
batch_size = 256


dataroot = "/home/shared-data/celeba/images"
dataset = datasets.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(img_size),
                               transforms.CenterCrop(img_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=10)

# Decide which device we want to run on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Hyperparameter

In der nächsten Zelle wählen Sie Hyperparameter wie die Dimension des latenten Raums oder die Zahl der Features für Generator und Diskriminator.

Bei GANs ist es ungünstig, wenn der Diskriminator deutlich besser ist als der Generator. Die Zahl der Features bzw. das Verhältnis zwischen Generator und Diskriminator kann hier einen Einfluss haben.

In [ ]:
#Size of latent vector
nz = 100

# Filter size of generator
ngf = 64

# Filter size of discriminator
ndf = 64

# Output image channels
nc = 3

## Initialisierung der Gewichte

Bei GANs ist die Initialisierung der Gewichte in den Netzwerken wichtig. Die folgende Funktion basiert auf Best Practices. 

In [ ]:
def weights_inititialisation(m):
    class_name = m.__class__.__name__
    if class_name.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif class_name.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

## Generator-Netzwerk

Definieren Sie hier das Generator-Netzwerk wie folgt:

- Verwenden Sie fünf `ConvTranspose2d` Schichten. Dabei sollen die Kanäle von `nz -> 8*ngf -> 4*ngf -> 2*ngf -> ngf -> nc` gewechselt werden.
- Die Kernelgröße ist jeweils 4.
- Die erste Schicht hat Stride 1 und Padding 0, die folgenden Schichten Stride 2 und Padding 1.
- Auf die ersten vier Schichten folgen jeweils `BatchNorm2d` und `ReLU`. 
- Die letzte Schicht verwendet als Aktivierungsfunktion `nn.Tanh`

In [ ]:
class _net_generator(nn.Module):
    def __init__(self):
        super(_net_generator, self).__init__()

        self.main = nn.Sequential(
         ### YOUR CODE HERE
        )

    def forward(self, input):
        output = self.main(input)
        return output


net_generator = _net_generator()
net_generator.apply(weights_inititialisation)
print(net_generator)

## Diskriminator-Netzwerk

Definieren Sie hier das Diskriminator-Netzwerk wie folgt:

- Verwenden Sie fünf `Conv2d` Schichten. Dabei sollen die Kanäle von `nc -> ndf -> 2*ndf -> 4*ndf -> 8*ndf` gewechselt werden.
- Die Kernelgröße ist jeweils 4.
- Die ersten vier Schichten haben Stride 2 und Padding 1, die letzte Schicht Stride 1 und Padding 0.
- Auf die ersten vier Schichten folgen jeweils `BatchNorm2d` und `LeakyReLU(0.2, inplace=True)`. 
- Die letzte Schicht verwendet als Aktivierungsfunktion `nn.Sigmoid`

In [ ]:
class _net_discriminator(nn.Module):
    def __init__(self):
        super(_net_discriminator, self).__init__()
        self.main = nn.Sequential(
            ### YOUR CODE HERE
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1).squeeze(1)


net_discriminator = _net_discriminator()
net_discriminator.apply(weights_inititialisation)
print(net_discriminator)

## Definition der Verlustfunktion und einiger Hilfswerte

Als Verlust verwenden wir die binäre Kreuzentropie (Warum?).

Der `fixed_noise` dient dazu, während des Trainings Bilder mit immer gleichen Startvektoren zu erzeugen.

In [ ]:
criterion = ### YOUR CODE HERE

input = torch.FloatTensor(batch_size, 3, img_size, img_size)
noise = torch.FloatTensor(batch_size, nz, 1, 1)
fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)
label = torch.FloatTensor(batch_size)
real_label = 1
fake_label = 0

In [ ]:
if torch.cuda.is_available():
    net_discriminator.cuda()
    net_generator.cuda()
    criterion.cuda()
    input, label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

## Definition des Optimizers

In [ ]:
lr = 0.0002
beta1 = 0.5
fixed_noise = Variable(fixed_noise)

optimizer_discriminator = optim.Adam(net_discriminator.parameters(), lr, betas=(beta1, 0.95))
optimizer_generator = optim.Adam(net_generator.parameters(), lr, betas=(beta1, 0.95))

## Trainingsschleife 

Die Trainingsschleife trainiert je Epoche

- zunächst den Diskriminator, der dazu je einen "echten" und einen "fake" Batch verarbeitet,
- dann den Generator. Hierbei ist zu beachten, dass er möglichst "echte" Bilder erzeugen soll und der Verlust daher gegen "real" Label gemessen wird.

In [ ]:
# Training Loop

num_epochs = 10

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch

with tqdm(range(num_epochs)) as pbar: 
    for epoch in pbar:
        # For each batch in the dataloader
        for i, (data, label) in enumerate(tqdm(dataloader), 0):

            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            ## Train with all-real batch
            net_discriminator.zero_grad()
            # Format batch
            real_cpu = data.to(device)
            b_size = real_cpu.size(0)
            label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
            # Forward pass real batch through D
            output = net_discriminator(real_cpu).view(-1)
            # Calculate loss on all-real batch
            errD_real = criterion(output, label)
            # Calculate gradients for D in backward pass
            errD_real.backward()
            D_x = output.mean().item()

            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            # Generate fake image batch with G
            fake = net_generator(noise)
            label.fill_(fake_label)
            # Classify all fake batch with D
            output = net_discriminator(fake.detach()).view(-1)
            # Calculate D's loss on the all-fake batch
            errD_fake = criterion(output, label)
            # Calculate the gradients for this batch, accumulated (summed) with previous gradients
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            # Compute error of D as sum over the fake and the real batches
            errD = errD_real + errD_fake
            # Update D
            optimizer_discriminator.step()

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            net_generator.zero_grad()
            label.fill_(real_label)  # fake labels are real for generator cost
            # Since we just updated D, perform another forward pass of all-fake batch through D
            output = net_discriminator(fake).view(-1)
            # Calculate G's loss based on this output
            errG = criterion(output, label)
            # Calculate gradients for G
            errG.backward()
            D_G_z2 = output.mean().item()
            # Update G
            optimizer_generator.step()

            # Output training stats
            if i % 50 == 0:
                pbar.set_postfix({ "Loss_D": errD.item(), "Loss_G": errG.item(), "D(x)": D_x, "D(G(z1))": D_G_z1, "D(G(z2))": D_G_z2})
            #    print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
            #          % (epoch, num_epochs, i, len(dataloader),
            #             errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

            # Save Losses for plotting later
            G_losses.append(errG.item())
            D_losses.append(errD.item())

            # Check how the generator is doing by saving G's output on fixed_noise
            if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
                with torch.no_grad():
                    fake = net_generator(fixed_noise).detach().cpu()
                img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

            iters += 1

## Darstellung der Ergebnisse

Die folgenden beiden Zellen plotten die Lernkurven und animieren die aus dem `fixed_noise` generierten Bilder.

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())